In [2]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_excel('./dataset/PTR_Rilis_D.xlsx', skiprows=9, sheet_name='Prototype - Regresi PTR Tester')
df.drop(columns='No', inplace=True)

In [6]:
df.rename(columns={'Sub Fitur': 'Sub-features', 'Rekening Sumber\n[Jika ada]' : 'Rekening Sumber', 'Data yang Digunakan\n[Jika ada]' : 'Data yang digunakan',
                   'FT\n[Jika Ada]' : 'FT', 'Tanggal Eksekusi\n[harus diisi]' : 'Tanggal Eksekusi', 'Tanggal Passed\n[harus diisi]' : 'Tanggal Passed'}, inplace=True)

In [7]:
df[['Features', 'Sub-features', 'Expected Condition']] = df[['Features', 'Sub-features', 'Expected Condition']].apply(lambda x: x.ffill())

In [8]:
df

,Features,Sub-features,Expected Condition,OS,OS Version,Tipe Device HP,Telko Provider HP,Rekening Sumber,Data yang digunakan,Tester,PO,FT,Status,Link Report Test,Description Issue dan Evidence,Tanggal Eksekusi,Tanggal Passed,Komentar
0,Aktivasi Registrasi (With Debit Card),Aktivasi Regresi,Berhasil Aktivasi dan masuk ke dashboard,Android,14,Pixel 8 Pro,NaN,NaN,NaN,Rezki,NaN,NaN,Passed,NaN,NaN,NaT,NaT,NaN
1,Aktivasi Registrasi (With Debit Card),Aktivasi Regresi,Berhasil Aktivasi dan masuk ke dashboard,iOS,17.6.1,Iphone 12 Pro Max,NaN,NaN,NaN,Hansen,NaN,NaN,Passed,NaN,NaN,NaT,NaT,NaN
2,Login Other Device,Login Other Device Normal Flow,Berhasil Login Dengan Normal Flow,Android,14,POCO F6,NaN,NaN,NaN,Haekal,NaN,NaN,Passed,NaN,NaN,NaT,NaT,NaN
3,Login Other Device,Login Other Device Normal Flow,Berhasil Login Dengan Normal Flow,iOS,17.6.1,Iphone 14,NaN,NaN,NaN,Rizky,NaN,NaN,Passed,NaN,NaN,NaT,NaT,NaN
4,Login Other Device,Login Other Device Lupa Password,Berhasil Login dengan kondisi Lupa Password,Android,14,POCO F6,NaN,NaN,NaN,Haekal,NaN,NaN,Passed,NaN,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,PLN,Postpaid (Pascabayar)\n\n,Berhasil Melakukan Pembayaran PLN Postpaid\n- ...,Android,14,Samsung Galaxy S22,NaN,7.203214e+09,515401131835\n513140591817\n513120565822,Nabhan,NaN,FT4311DC8BX,Passed,NaN,NaN,NaT,NaT,NaN
130,PLN,Postpaid (Pascabayar)\n\n,Berhasil Melakukan Pembayaran PLN Postpaid\n- ...,iOS,17.5.1,Iphone 13,NaN,7.202088e+09,522601398196\n514530958367\n513140695457,Densu,NaN,NaN,Passed,NaN,NaN,NaT,NaT,NaN
131,Tarik Tunai tanpa kartu,Tarik Tunai Tanpa Kartu,Berhasil Melakukan Tarik Tunai Tanpa Kartu\n- ...,Android,11,Redmi Note 12 Pro,Telkomsel,7.200546e+09,81293394417,Owan,NaN,NaN,Passed,NaN,NaN,NaT,NaT,NaN
132,Tarik Tunai tanpa kartu,Tarik Tunai Tanpa Kartu,Berhasil Melakukan Tarik Tunai Tanpa Kartu\n- ...,Android,14,Poco F4,Telkomsel,7.200546e+09,81293394417,Owan,NaN,NaN,Passed,NaN,NaN,NaT,NaT,NaN


### Nyoba Sankey Diagram

In [10]:
import pandas as pd
import plotly.graph_objects as go

# Extended dummy dataset
data = {
    "Sub-Feature": ["Activation", "Login", "Activation", "Payment", "Login", "Payment"],
    "Feature": ["Registration", "Authentication", "Registration", "Transactions", "Authentication", "Transactions"],
    "OS": ["Android", "iOS", "Android", "iOS", "Android", "iOS"],
    "OS Version": ["14", "17.6.1", "13", "16.3", "12", "15.2"],
    "Status": ["Passed", "Failed", "Passed", "Failed", "Failed", "Passed"],
    "Tester": ["Rezki", "Hansen", "Alex", "Maria", "John", "Sarah"],
    "Count": [5, 3, 7, 4, 6, 2]  # Example count values
}
df = pd.DataFrame(data)

# Create nodes and flows
nodes = list(set(
    df["Sub-Feature"].tolist() +
    df["Feature"].tolist() +
    df["OS"].tolist() +
    df["OS Version"].tolist() +
    df["Status"].tolist() +
    df["Tester"].tolist()
))
node_indices = {node: i for i, node in enumerate(nodes)}

# Create flows (source → target)
sources = (
    [node_indices[sub_feature] for sub_feature in df["Sub-Feature"]] +  # Sub-Feature → Feature
    [node_indices[feature] for feature in df["Feature"]] +              # Feature → OS
    [node_indices[os] for os in df["OS"]] +                             # OS → OS Version
    [node_indices[os_version] for os_version in df["OS Version"]] +     # OS Version → Status
    [node_indices[status] for status in df["Status"]]                   # Status → Tester
)
targets = (
    [node_indices[feature] for feature in df["Feature"]] +              # Sub-Feature → Feature
    [node_indices[os] for os in df["OS"]] +                             # Feature → OS
    [node_indices[os_version] for os_version in df["OS Version"]] +     # OS → OS Version
    [node_indices[status] for status in df["Status"]] +                 # OS Version → Status
    [node_indices[tester] for tester in df["Tester"]]                   # Status → Tester
)

# Values for each flow
values = df["Count"].tolist() * 5  # Repeat counts for each layer of connection

# Define colors for each node
node_colors = {
    "Activation": "rgba(102, 194, 165, 0.8)",
    "Login": "rgba(252, 141, 98, 0.8)",
    "Payment": "rgba(141, 160, 203, 0.8)",
    "Registration": "rgba(231, 138, 195, 0.8)",
    "Authentication": "rgba(166, 216, 84, 0.8)",
    "Transactions": "rgba(255, 217, 47, 0.8)",
    "Android": "rgba(31, 119, 180, 0.8)",
    "iOS": "rgba(255, 127, 14, 0.8)",
    "Rezki": "rgba(44, 160, 44, 0.8)",
    "Hansen": "rgba(214, 39, 40, 0.8)",
    "Alex": "rgba(140, 86, 75, 0.8)",
    "Maria": "rgba(227, 119, 194, 0.8)",
    "John": "rgba(188, 189, 34, 0.8)",
    "Sarah": "rgba(127, 127, 127, 0.8)",
    "Passed": "rgba(148, 103, 189, 0.8)",
    "Failed": "rgba(227, 119, 194, 0.8)",
    "14": "rgba(255, 215, 0, 0.8)",
    "17.6.1": "rgba(0, 191, 255, 0.8)",
    "13": "rgba(124, 252, 0, 0.8)",
    "16.3": "rgba(70, 130, 180, 0.8)",
    "12": "rgba(220, 20, 60, 0.8)",
    "15.2": "rgba(0, 250, 154, 0.8)"
}

# Generate colors for nodes
node_color_list = [node_colors[node] for node in nodes]

# Sankey Diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes,
        color=node_color_list
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color="rgba(192, 192, 192, 0.4)"  # Optional: Adjust link color
    )
)])

# Update layout and show
fig.update_layout(
    title_text="Sankey Diagram: Sub-Feature → Feature → OS → OS Version → Status → Tester",
    font_size=12
)
fig.show()


### Implementasi

In [23]:
df.columns

Index(['Features', 'Sub-features', 'Expected Condition', 'OS', 'OS Version',
       'Tipe Device HP', 'Telko Provider HP', 'Rekening Sumber',
       'Data yang digunakan', 'Tester', 'PO', 'FT', 'Status',
       'Link Report Test', 'Description Issue dan Evidence',
       'Tanggal Eksekusi', 'Tanggal Passed', 'Komentar'],
      dtype='object')

In [ ]:
import pandas as pd
import plotly.graph_objects as go

nodes = list(set(df['Features'].tolist() +
                 df['Sub-features'].tolist() +
                 df['OS'].tolist() +
                 df['OS Version'].tolist() +
                 df['Tipe Device HP'].tolist() +
                 df['Tester'].tolist() +
                 df['Status'].tolist()
                 ))



['Login Other Device',
 'Login Other Device Lupa Password',
 11,
 13,
 14,
 16.6,
 17.5,
 'Fira',
 'Lupa Password Dari Prelogin',
 'Lokasi ATM & Cabang',
 '17.6.1',
 'Lokasi Masjid terdekat',
 'MPM ON Us (Payment)',
 'Owan',
 'Asmaul Husna',
 'Rekening Di Homescreen',
 'Lupa PIN',
 'Hansen',
 'Natasya',
 'QRIS',
 'Sanggahan Transaksi',
 'Tambah tabungan',
 'Topup Gopay',
 'Topup Ovo',
 'Mutasi Rekening',
 'Juzamma',
 'Sesama BSI dari Favorit',
 'Poco F4',
 'Prita',
 'QRIS CPM (Payment)',
 'Aktivasi Regresi',
 'POCO F6',
 'Informasi Rekening',
 'Kintan',
 'Arah Kiblat',
 'Layanan Islami',
 'Tarik Tunai Tanpa Kartu',
 'Profil',
 'Login Other Device Lupa PIN',
 'Login Other Device Normal Flow',
 'Lupa Password dari menu profil Kemudian lupa PIN',
 'Sanggahan dari halaman mutasi',
 'Infaq After Transaksi',
 'Hanifah',
 'List & Icon Bayar Beli',
 'Iphone 14 Pro Max',
 'Sesama BSI dari History',
 'Nabhan',
 'Prelogin Card Waktu Sholat (Lokasi, Saat Adzan, Sebelum / setelah Adzan)',
 'Prelogi

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px  # For color palettes

# Example: Load your actual data into `df`
# df = pd.read_csv("your_data.csv")

# Create nodes and flows
nodes = list(set(
    df['Features'].tolist() +
    df['Sub-features'].tolist() +
    df['OS'].tolist() +
    df['OS Version'].tolist() +
    df['Tipe Device HP'].tolist() +
    df['Status'].tolist() +
    df['Tester'].tolist()
))
node_indices = {node: i for i, node in enumerate(nodes)}

# Create flows (source → target)
sources = (
    [node_indices[feature] for feature in df["Features"]] +            # Features → Sub-features
    # [node_indices[sub_feature] for sub_feature in df["Sub-features"]] + # Sub-features → Status
    [node_indices[status] for status in df["Status"]] +                # Status → OS
    [node_indices[os] for os in df["OS"]]                              # OS → OS Version
)
targets = (
    # [node_indices[sub_feature] for sub_feature in df["Sub-features"]] + # Features → Sub-features
    [node_indices[status] for status in df["Status"]] +                # Sub-features → Status
    [node_indices[os] for os in df["OS"]]                              # Status → OS
)

# Assign a default value of 1 for each connection
values = [1] * len(sources)

# Use a qualitative color scale for high contrast (e.g., Plotly's 'Dark24')
color_palette = px.colors.qualitative.Light24 # Replace with another palette if needed
num_colors = len(color_palette)

# Assign contrasting colors to features
node_colors = {}
for i, feature in enumerate(df["Features"].unique()):
    node_colors[feature] = color_palette[i % num_colors]  # Cycle through the color palette

# Propagate feature colors to sub-features
# for feature in df["Features"].unique():
#     feature_color = node_colors[feature]
#     for sub_feature in df[df["Features"] == feature]["Sub-features"].unique():
#         node_colors[sub_feature] = feature_color

# Overwrite colors for 'Passed' and 'Failed'
node_colors["Passed"] = "rgba(144, 238, 144, 0.8)"  # Soft green
node_colors["Failed"] = "rgba(205, 92, 92, 0.8)"    # Soft red brick

node_colors["Android"] = "rgba(64, 224, 208, 0.8)"  # Green turquoise
node_colors["iOS"] = "rgba(70, 130, 180, 0.8)"      # Steel blue

# Set default color for unassigned nodes
default_color = "rgba(200, 200, 200, 0.8)"
node_color_list = [node_colors.get(node, default_color) for node in nodes]

# Sankey Diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes,
        color=node_color_list
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,  # Default value of 1 for each connection
        color="rgba(192, 192, 192, 0.4)"  # Optional: Adjust link color
    )
)])

# Update layout and show
fig.update_layout(
    title_text="Sankey Diagram: Features → Sub-features → Status → OS → OS Version",
    font_size=12,
    width=1400,  # Increase width for a wider graph
    height=1500,   # Increase height for a taller graph
    font=dict(size=10, color='white'),
    plot_bgcolor='black',
    paper_bgcolor='black'
)
fig.show()


In [76]:
df = df.groupby(['Features', 'Sub-features', 'Status'], as_index=False).size()
df

,Features,Sub-features,Status,size
0,Aktivasi Registrasi (With Debit Card),Aktivasi Regresi,Passed,2
1,Bayar Beli,List & Icon Bayar Beli,Passed,2
2,Bayar Beli,Routing Menu pada Bayarbeli,Passed,2
3,Berbagi,Bayar Infaq,Passed,2
4,Berbagi,Bayar Zakat,Passed,2
...,...,...,...,...
61,Transfer,Online Syariah\n- Android Maybank Syariah\n- i...,Passed,2
62,Transfer,Sesama BSI dari Favorit,Passed,2
63,Transfer,Sesama BSI dari History,Passed,2
64,Transfer,Sesama BSI dari tambah baru,Passed,2


In [37]:
nodes

['Login Other Device',
 'Login Other Device Lupa Password',
 11,
 13,
 14,
 16.6,
 17.5,
 'Fira',
 'Lupa Password Dari Prelogin',
 'Lokasi ATM & Cabang',
 '17.6.1',
 'Lokasi Masjid terdekat',
 'MPM ON Us (Payment)',
 'Owan',
 'Asmaul Husna',
 'Rekening Di Homescreen',
 'Lupa PIN',
 'Hansen',
 'Natasya',
 'QRIS',
 'Sanggahan Transaksi',
 'Tambah tabungan',
 'Topup Gopay',
 'Topup Ovo',
 'Mutasi Rekening',
 'Juzamma',
 'Sesama BSI dari Favorit',
 'Poco F4',
 'Prita',
 'QRIS CPM (Payment)',
 'Aktivasi Regresi',
 'POCO F6',
 'Informasi Rekening',
 'Kintan',
 'Arah Kiblat',
 'Layanan Islami',
 'Tarik Tunai Tanpa Kartu',
 'Profil',
 'Login Other Device Lupa PIN',
 'Login Other Device Normal Flow',
 'Lupa Password dari menu profil Kemudian lupa PIN',
 'Sanggahan dari halaman mutasi',
 'Infaq After Transaksi',
 'Hanifah',
 'List & Icon Bayar Beli',
 'Iphone 14 Pro Max',
 'Sesama BSI dari History',
 'Nabhan',
 'Prelogin Card Waktu Sholat (Lokasi, Saat Adzan, Sebelum / setelah Adzan)',
 'Prelogi

In [70]:
import pandas as pd

# Load the Excel file
df = pd.read_excel('./dataset/PTR_Rilis_D.xlsx', sheet_name='Prototype - Regresi PTR Tester')

# Value to find and use as new header
value_to_skip = 'Features'

# Find the index of the row containing the value
header_index = df.apply(lambda row: row.astype(str).str.contains(value_to_skip).any(), axis=1).idxmax()

# Extract the header from the found row and create a new DataFrame
new_header = df.iloc[header_index].values
df = df.iloc[header_index+1:].copy()

# Set the new header
df.columns = new_header

# Drop the first column dynamically
df = df.iloc[:, 1:]

# Reset the index without adding the old index as a column
df.reset_index(drop=True, inplace=True)

df


,Features,Sub Fitur,Expected Condition,OS,OS Version,Tipe Device HP,Telko Provider HP,Rekening Sumber\n[Jika ada],Data yang Digunakan\n[Jika ada],Tester,PO,FT\n[Jika Ada],Status,Link Report Test,Description Issue dan Evidence,Tanggal Eksekusi\n[harus diisi],Tanggal Passed\n[harus diisi],Komentar
0,Aktivasi Registrasi (With Debit Card),Aktivasi Regresi,Berhasil Aktivasi dan masuk ke dashboard,Android,14,Pixel 8 Pro,NaN,NaN,NaN,Rezki,NaN,NaN,Passed,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,iOS,17.6.1,Iphone 12 Pro Max,NaN,NaN,NaN,Hansen,NaN,NaN,Passed,NaN,NaN,NaN,NaN,NaN
2,Login Other Device,Login Other Device Normal Flow,Berhasil Login Dengan Normal Flow,Android,14,POCO F6,NaN,NaN,NaN,Haekal,NaN,NaN,Passed,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,iOS,17.6.1,Iphone 14,NaN,NaN,NaN,Rizky,NaN,NaN,Passed,NaN,NaN,NaN,NaN,NaN
4,NaN,Login Other Device Lupa Password,Berhasil Login dengan kondisi Lupa Password,Android,14,POCO F6,NaN,NaN,NaN,Haekal,NaN,NaN,Passed,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,NaN,Postpaid (Pascabayar)\n\n,Berhasil Melakukan Pembayaran PLN Postpaid\n- ...,Android,14,Samsung Galaxy S22,NaN,7203213942,515401131835\n513140591817\n513120565822,Nabhan,NaN,FT4311DC8BX,Passed,NaN,NaN,NaN,NaN,NaN
130,NaN,NaN,NaN,iOS,17.5.1,Iphone 13,NaN,7202088017,522601398196\n514530958367\n513140695457,Densu,NaN,NaN,Passed,NaN,NaN,NaN,NaN,NaN
131,Tarik Tunai tanpa kartu,Tarik Tunai Tanpa Kartu,Berhasil Melakukan Tarik Tunai Tanpa Kartu\n- ...,Android,11,Redmi Note 12 Pro,Telkomsel,7200545588,81293394417,Owan,NaN,NaN,Passed,NaN,NaN,NaN,NaN,NaN
132,NaN,NaN,NaN,Android,14,Poco F4,Telkomsel,7200545588,81293394417,Owan,NaN,NaN,Passed,NaN,NaN,NaN,NaN,NaN


In [86]:
import pandas as pd

# Load the Excel file
file_path = './dataset/PTR_Rilis_D.xlsx'
specific_value = 'PTR Tester'

# Load Excel file and get sheet names
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names

# Initialize variable to store the target sheet
target_sheet = None

# Iterate through the sheet names to find the specific value
for sheet in sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)
    if specific_value in sheet or df.apply(lambda row: row.astype(str).str.contains(specific_value, case=False, na=False)).any().any():
        target_sheet = sheet
        break

# Check if the target sheet was found
if target_sheet:
    df = pd.read_excel(file_path, sheet_name=target_sheet)
    print(f"Found '{specific_value}' in sheet: {target_sheet}")
    print(df)  # Display the first few rows of the DataFrame
else:
    print(f"Value '{specific_value}' not found in any sheet.")

# Display the target sheet DataFrame
df


Found 'PTR Tester' in sheet: Prototype - Regresi PTR Tester
    Unnamed: 0            Judul / Envi : Regresi / PTR BYOND  \
0          NaN                                       Version   
1          NaN  PTR Ver Android 1.0.3 (272)\niOS 1.0.3 (267)   
2          NaN                                           NaN   
3          NaN                                           NaN   
4          NaN                                           NaN   
..         ...                                           ...   
138         64                                           NaN   
139        NaN                                           NaN   
140         65                       Tarik Tunai tanpa kartu   
141        NaN                                           NaN   
142        NaN                                           NaN   

    Version : Android 1.0.2 (270) / iOS 1.0.2 (238)  \
0                                                OS   
1                                           Android   
2     

,Unnamed: 0,Judul / Envi : Regresi / PTR BYOND,Version : Android 1.0.2 (270) / iOS 1.0.2 (238),Periode Testing : 5 November 2024,Unnamed: 4,Link Upload Capture,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,NaN,Version,OS,Total Fitur,Total Sub Fitur,Execution,NaN,Passed,NaN,Failed,NaN,NaN,NaN,In Progress,NaN,NaN,NaN,NaN,NaN
1,NaN,PTR Ver Android 1.0.3 (272)\niOS 1.0.3 (267),Android,19,65,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,NaN,NaN,iOS,19,65,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,o,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,64,NaN,Postpaid (Pascabayar)\n\n,Berhasil Melakukan Pembayaran PLN Postpaid\n- ...,Android,14,Samsung Galaxy S22,NaN,7203213942,515401131835\n513140591817\n513120565822,Nabhan,NaN,FT4311DC8BX,Passed,NaN,NaN,NaN,NaN,NaN
139,NaN,NaN,NaN,NaN,iOS,17.5.1,Iphone 13,NaN,7202088017,522601398196\n514530958367\n513140695457,Densu,NaN,NaN,Passed,NaN,NaN,NaN,NaN,NaN
140,65,Tarik Tunai tanpa kartu,Tarik Tunai Tanpa Kartu,Berhasil Melakukan Tarik Tunai Tanpa Kartu\n- ...,Android,11,Redmi Note 12 Pro,Telkomsel,7200545588,81293394417,Owan,NaN,NaN,Passed,NaN,NaN,NaN,NaN,NaN
141,NaN,NaN,NaN,NaN,Android,14,Poco F4,Telkomsel,7200545588,81293394417,Owan,NaN,NaN,Passed,NaN,NaN,NaN,NaN,NaN


In [ ]:
import streamlit as st
import pandas as pd
from st_aggrid import AgGrid, GridOptionsBuilder, JsCode

def display_tester_page():
    with st.container():
        col1, col2 = st.columns([1, 1])
        
        with col1:
            st.subheader('Dashboard')
            skip_rows = st.number_input('Enter the number of rows to skip', min_value=0, value=0, step=1)
            select_sheet = st.text_input("Enter the name of the sheet")
            data_uploaded = st.file_uploader('Choose an excel file', type='xlsx')
            
        with col2:
            st.header(' ')
    
    if data_uploaded is not None and select_sheet:
        try:
            # Read Excel file
            excel_ptr = pd.read_excel(data_uploaded, skiprows=skip_rows, sheet_name=select_sheet, dtype={'YourColumnName': str})
            excel_ptr.drop(columns='No', inplace=True)
            excel_ptr.rename(columns={
                'Sub Fitur': 'Sub-features',
                'Rekening Sumber\n[Jika ada]': 'Rekening Sumber',
                'Data yang Digunakan\n[Jika ada]': 'Data yang digunakan',
                'FT\n[Jika Ada]': 'FT',
                'Tanggal Eksekusi\n[harus diisi]': 'Tanggal Eksekusi',
                'Tanggal Passed\n[harus diisi]': 'Tanggal Passed'
            }, inplace=True)
            excel_ptr[['Features', 'Sub-features', 'Expected Condition']] = excel_ptr[['Features', 'Sub-features', 'Expected Condition']].apply(lambda x: x.ffill())

            # Set up the GridOptionsBuilder with enhancements
            gb = GridOptionsBuilder.from_dataframe(excel_ptr)
            gb.configure_default_column(resizable=True, filterable=True, sortable=True, editable=True, cellStyle={'padding': '10px', 'borderBottom': '1px solid #ddd', 'fontSize':'20px'})
            gb.configure_column("Features", rowGroup=True, hide=True)
            gb.configure_column("Sub-features", rowGroup=True, hide=True)
            gb.configure_column("Expected Condition", rowGroup=True, hide=True, tooltipField="Expected Condition", headerTooltip="Expected Condition")

            # Conditional formatting for critical items in Sub-features
            gb.configure_column("Sub-features", cellStyle=JsCode("""
                function(params) {
                    if (params.value && params.value.includes("Critical")) {
                        return { 'backgroundColor': '#FFDDC1', 'color': '#B22222', 'font-weight': 'bold' };
                    }
                    return {};
                }
            """))

            # Increase row height and set other grid options
            gridOptions = gb.build()
            gridOptions['rowHeight'] = 40  # Increase row height
            gridOptions['groupDefaultExpanded'] = -1  # Expand all groups by default
            gridOptions['suppressColumnVirtualisation'] = True
            gridOptions['groupDisplayType'] = "groupRows"
            gridOptions['autoGroupColumnDef'] = {
                'minWidth': 150, 
                'pinned': "left", 
                'cellRendererParams': {'suppressCount': True}
            }

            # Display the dataframe with the "balham" theme
            AgGrid(excel_ptr, gridOptions=gridOptions, height=600, theme="balham", allow_unsafe_jscode=True)
                    
        except Exception as e:
            st.error(f"An error occurred: {e}")

if __name__ == "__main__":
    display_tester_page()


### Testing upload file di gdrive

In [9]:
from googleapiclient.discovery import build
from google.oauth2 import service_account

SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'service_account.json'
PARENT_FOLDER_ID = '1vIKk9xgUn1JGY2MFPdB6W3tEfU8oVptP'

def authenticate():
    creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    return creds

def upload_file(file_path):
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)
    
    file_metadata = {
        'name':'PTR_Rilis_D',
        'parents':[PARENT_FOLDER_ID]
    }
    
    file = service.files().create(
        body=file_metadata,
        media_body=file_path
    ).execute()
    
upload_file('./dataset/PTR_Rilis_D.xlsx')

In [ ]:
import pandas as pd
from googleapiclient.discovery import build
from google.oauth2 import service_account
from io import BytesIO
from googleapiclient.http import MediaIoBaseDownload

SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'service_account.json'
PARENT_FOLDER_ID = '1SjXetNj-_yrXgs1rfIMM_ISmWdECBbic'

def authenticate():
    '''
    Fungsi untuk autentikasi google biar bisa akses gdrive
    '''
    creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    return creds

def read_file_from_drive(file_id):
    '''
    Baca list file excel dari gdrive
    '''
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)
    
    request = service.files().get_media(fileId=file_id)
    file_data = BytesIO()
    downloader = MediaIoBaseDownload(file_data, request)
    done = False
    while done is False:
        _, done = downloader.next_chunk()
    
    file_data.seek(0)
    df = pd.read_excel(file_data)
    return df

# List files in the folder
def list_files():
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)
    
    results = service.files().list(
        q=f"'{PARENT_FOLDER_ID}' in parents",
        spaces='drive',
        fields='nextPageToken, files(id, name)'
    ).execute()
    
    items = results.get('files', [])
    
    if not items:
        print('No files found.')
    else:
        for item in items:
            print(f"{item['name']}")
    return items

files = list_files()

User
PTR


## Testing chart that fits to problem

In [20]:
# df[['OS', 'Status']]

[(x, y) for x, y in zip(df['OS'].values, df['Status'].values)]

[('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS', 'Passed'),
 ('Android', 'Passed'),
 ('iOS',

In [24]:
# hitung progress execution rate, passed, dll

listof_status = [(x, y) for x, y in zip(df['OS'].values, df['Status'].values)]

passed = sum(1 for i in listof_status if i == ('Android', 'Passed'))
passed

65

In [3]:
import plotly.graph_objects as go

# Progress value (you can dynamically update this)
progress = 75

# Create a progress bar chart
fig = go.Figure(go.Bar(
    x=[progress, 100-progress],
    y=["Progress"],
    orientation='h',
    marker=dict(color=['green', 'lightgray'], line=dict(width=1))
))

# Customize layout
fig.update_layout(
    xaxis=dict(range=[0, 100], showticklabels=False),
    yaxis=dict(showticklabels=False),
    showlegend=False,
    title="Progress to 100%",
    margin=dict(l=20, r=20, t=30, b=20),
    height=200
)

# Display in Streamlit
fig


In [1]:
b=['a','b','c','d','e','f','g','h','i']
def testing(a = b):
    abc = print(a)
    return abc

In [4]:
testing(a = [1,2,3,4])

[1, 2, 3, 4]


In [2]:
testing()

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']


## Testing to get list of features

In [77]:
df = pd.read_excel('./dataset/PTR_Nov_2024.xlsx', sheet_name='Regresi PTR Tester (2)', header=None)

In [78]:
temp_df = df.ffill()

In [79]:
def get_ptr_versions(temp_df):
    listof_ver = temp_df[temp_df[1].str.contains('PTR Ver')][1].unique().tolist()
    if not listof_ver:  # Check if the list is empty
        return None
    return listof_ver

In [81]:
df

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,NaN,Judul / Envi : Regresi / PTR BYOND,Version : APK 1.0.3 (275) dan IPA 1.0.3 (282) ...,Periode Testing : 24 November 2024,NaN,Link Upload Capture,https://bsicenter-my.sharepoint.com/:f:/g/pers...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,OS,Total Fitur,Total Sub Fitur,Execution,NaN,Passed,NaN,Failed,...,In Progress,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PTR,PTR prev. version\nAndroid 1.0.3 (275)\niOS 1....,Android,19,64,64,1,64,1,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,iOS,19,64,63,0.984375,63,0.984375,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,No,Features,Sub Fitur,Expected Condition,OS,OS Version,Tipe Device HP,Telko Provider HP,Rekening Sumber\n[Jika ada],Data yang Digunakan\n[Jika ada],...,Status PTR\nAPK 1.0.3 (275) \nIPA 1.0.3 (282),Link Report Test,Description Issue dan Evidence,Tanggal Eksekusi\n[harus diisi],Tanggal Passed\n[harus diisi],Tanggal Eksekusi\n[harus diisi],Tanggal Passed\n[harus diisi],Tanggal Eksekusi\n[harus diisi],Tanggal Passed\n[harus diisi],Komentar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,NaN,NaN,NaN,NaN,iOS,18.1,Iphone 15 pro,NaN,7243447776,IDPEL 566602186874,...,Passed,NaN,NaN,2024-11-24 00:00:00,2024-11-24 00:00:00,NaN,NaN,2024-11-15 00:00:00,2024-11-15 00:00:00,NaN
129,64,NaN,Postpaid (Pascabayar)\n\n,Berhasil Melakukan Pembayaran PLN Postpaid\n- ...,Android,14,Xiaomi 14,NaN,NaN,539112105280\n521511299379\n518040539319\n5220...,...,Passed,NaN,NaN,2024-11-15 00:00:00,2024-11-15 00:00:00,NaN,NaN,NaN,NaN,NaN
130,NaN,NaN,NaN,NaN,iOS,17.5.1,Iphone 13,NaN,NaN,539112105280\n521511299379\n518040539319\n5220...,...,Passed,NaN,NaN,2024-11-15 00:00:00,2024-11-15 00:00:00,NaN,NaN,NaN,NaN,NaN
131,65,Tarik Tunai tanpa kartu,Tarik Tunai Tanpa Kartu,Berhasil Melakukan Tarik Tunai Tanpa Kartu\n- ...,Android,14,Poco F4,NaN,7200545588,NaN,...,Passed,NaN,NaN,2024-11-15 00:00:00,2024-11-15 00:00:00,NaN,NaN,2024-11-15 00:00:00,2024-11-15 00:00:00,NaN


In [ ]:
get_ptr_versions(temp_df)

In [77]:
value_to_skip = 'Features'
header_index = df.apply(lambda row: row.astype(str).str.contains(value_to_skip).any(), axis=1).idxmax()
new_header = df.iloc[header_index].values
df = df.iloc[header_index+1:].copy()

df.columns = new_header
df = df.iloc[:, 1:]
df.reset_index(drop=True, inplace=True)

In [78]:
df.head(4)

,Features,Sub Fitur,Expected Condition,OS,OS Ver,Tipe Device HP,Telko Provider HP,Rekening Sumber\n[Jika ada],Data yang Digunakan\n[Jika ada],PO,FT\n[Jika Ada],Status lama,Status baru PTR Ver Android 1.0.3 (272)\niOS 1.0.3 (267),Link Report Test,Description Issue dan Evidence,Tanggal Eksekusi\n[harus diisi],Tanggal Passed\n[harus diisi],Tanggal Eksekusi\n[harus diisi],Tanggal Passed\n[harus diisi],Komentar
0,PS-495 : RELEASE A - IOS - Tidak bisa klik ban...,Donasi,Byond dapat menampilkan halaman list LAZ Donasi,iOS,16.3,iPhone XS,Telkomsel,Rekening Sumber\n[Jika ada],Data yang Digunakan\n[Jika ada],"Iin, Dhimas",FT\n[Jika Ada],Passed,Not Started,Link Report Test,evidence :\n\nhttps://drive.google.com/drive/f...,2024-11-05 00:00:00,2024-11-05 00:00:00,Tanggal Eksekusi\n[harus diisi],Tanggal Passed\n[harus diisi],Scenario :\n1. Login Byond dengan iOS\n2. Klik...
1,PS-833 : IOS - TRF TERJADWAL SEKALI GAGAL,Donasi,Berhasil melakukan trf terjadwal dalam jangka ...,Android,13,Galaxy S20+,Telkomsel,Rekening Sumber\n[Jika ada],Data yang Digunakan\n[Jika ada],"Iin, Dhimas",FT\n[Jika Ada],Passed,Not Started,https://bsicenter-my.sharepoint.com/:v:/g/pers...,evidence :\n\nhttps://drive.google.com/drive/f...,2024-11-05 00:00:00,2024-11-05 00:00:00,Tanggal Eksekusi\n[harus diisi],Tanggal Passed\n[harus diisi],Scenario :\n1. Login Byond dengan iOS\n2. Klik...
2,PS-833 : IOS - TRF TERJADWAL SEKALI GAGAL,Donasi,Berhasil melakukan trf terjadwal dalam jangka ...,iOS,17.6.1,iPhone 12,Telkomsel,Rekening Sumber\n[Jika ada],Data yang Digunakan\n[Jika ada],"Iin, Dhimas",FT\n[Jika Ada],Passed,Not Started,https://bsicenter-my.sharepoint.com/:v:/g/pers...,evidence :\n\nhttps://drive.google.com/drive/f...,2024-11-05 00:00:00,2024-11-05 00:00:00,Tanggal Eksekusi\n[harus diisi],Tanggal Passed\n[harus diisi],Scenario :\n1. Login Byond dengan iOS\n2. Klik...
3,PS-817: RELEASE B- iOS & Android - Push Notif ...,Donasi,Berhasil melakukan trf terjadwal dalam jangka ...,Android,13,Samsung A51,Telkomsel,Rekening Sumber\n[Jika ada],No HP: 6281297976944,"Nauvalady, Ori, Inna",FT\n[Jika Ada],Failed,Not Started,https://bsicenter-my.sharepoint.com/:v:/g/pers...,Gagal direct ke halaman Promo,2024-11-05 00:00:00,2024-11-05 00:00:00,Tanggal Eksekusi\n[harus diisi],Tanggal Passed\n[harus diisi],Scenario :\n1. Login Byond dengan iOS\n2. Klik...


In [57]:
listof_ver

['PTR Ver Android 1.0.3 (272) \n iOS 1.0.3 (266)',
 'PTR Ver Android 1.0.3 (272)\niOS 1.0.3 (267)']

In [1]:
def processing_excel(file_data, sheet_name):
    '''
    Process Excel file to clean and prepare the data
    '''
    
    excel_ptr = pd.read_excel(file_data, sheet_name, header=None)
    test = excel_ptr.ffill()
    listof_ver = test[test[1].str.contains('PTR Ver', na=False)][1].unique().tolist()

    

    value_to_skip = 'Features'
    header_index = excel_ptr.apply(lambda row: row.astype(str).str.contains(value_to_skip).any(), axis=1).idxmax() # to find which index that contain 'Features'
    new_header = excel_ptr.iloc[header_index].values
    excel_ptr = excel_ptr.iloc[header_index+1:].copy()
    
    excel_ptr.columns = new_header
    excel_ptr = excel_ptr.iloc[:, 1:]
    excel_ptr.reset_index(drop=True, inplace=True)
    
    # # Convert column OS Version types
    # if 'OS Version' not in excel_ptr.columns:
    #     print('error')
    # else:
    #     excel_ptr['OS Version'] = excel_ptr['OS Version'].astype(str)
    
    # # drop column Number
    # excel_ptr.drop(columns='No', inplace=True, errors='ignore')
    
    # #Rename the column
    # excel_ptr.rename(columns={
    #     'Sub Fitur': 'Sub-features',
    #     'Rekening Sumber\n[Jika ada]': 'Rekening Sumber',
    #     'Data yang Digunakan\n[Jika ada]': 'Data yang digunakan',
    #     'FT\n[Jika Ada]': 'FT',
    #     'Tanggal Eksekusi\n[harus diisi]': 'Tanggal Eksekusi',
    #     'Tanggal Passed\n[harus diisi]': 'Tanggal Passed'
    # }, inplace=True)
    
    # # Because of merged and centered, need to use ffill to duplicate values before current rows
    # excel_ptr[['Features', 'Sub-features', 'Expected Condition']] = excel_ptr[['Features', 'Sub-features', 'Expected Condition']].apply(lambda x: x.ffill())
    
    return excel_ptr, listof_ver

In [2]:
path = './dataset/PTR_Rilis_D.xlsx'

In [3]:
new_df, listof_status= processing_excel(path, sheet_name='Prototype - Tiket Prod Issue PO')

NameError: name 'pd' is not defined